In [36]:
import csv
import itertools
import pandas as pd
import numpy as np
import networkx as nx

In [37]:
papers=[]
with open("train.txt", "r") as f:
    for x in f:
        token_list=x.rstrip("\n").split()
        papers.append(token_list)

graph=[]

for paper in papers:
    for pair in itertools.combinations(paper, 2):
        graph.append(pair)

graph_df=pd.DataFrame(graph, columns=['Author_A','Author_B'])
#Calculate weights
g_df = graph_df.groupby(['Author_A','Author_B']).size().reset_index().rename(columns={0:'count'})
#create undirectional graph 
ng = nx.convert_matrix.from_pandas_edgelist(g_df, source= 'Author_A', target = 'Author_B', edge_attr= 'count')
print(nx.info(ng))
print(g_df)

Name: 
Type: Graph
Number of nodes: 3767
Number of edges: 16036
Average degree:   8.5139
      Author_A Author_B  count
0            0     1236     14
1            0     1655      9
2            0     1797      4
3            0     2414      1
4            0     2568      8
...        ...      ...    ...
16031      999     2038      1
16032      999     2250      4
16033      999     2600      1
16034      999     2905      3
16035      999     3926      1

[16036 rows x 3 columns]


In [8]:
cnc = nx.common_neighbor_centrality(ng)

<generator object _apply_prediction.<locals>.<genexpr> at 0x000001FECE4DEF20>


In [9]:
[print (a,b,c) for a,b,c in cnc]

KeyError: '603'

In [ ]:
# this step takes ages Link prediction metrics are very slow to calculate
#cnc = nx.common_neighbor_centrality(ng)
feature_1 = []
for s,t in ng.edges():
    try:
        metric = nx.common_neighbor_centrality(ng, [(s,t)])
        for u,v,p in metric:
            feature_1.append(p)
    except:
        feature_1.append(0)

In [11]:
#getting all set of edges
r = csv.reader(open('no_headers_graph_pos.csv','r'))
edges = dict()
for edge in r:
    edges[(edge[0], edge[1])] = 1

In [ ]:
column = g_df["Author_A"]
max_value_a = column.max()

In [ ]:
column = g_df["Author_B"]
max_value_b = column.max()

In [12]:
%%time
###generating negative edges from given graph
import random
missing_edges = set([])
while (len(missing_edges)<16036): # such that same number as positive edges
    a=random.randint(1, 3767) # whereby the higher bound is the number of nodes of ng.
    b=random.randint(1, 3767)
    tmp = edges.get((a,b),-1)
    if tmp == -1 and a!=b:
        try:
            if nx.shortest_path_length(ng,source=a,target=b) > 2: 
                
                missing_edges.add((a,b))
            else:
                continue  
        except:  
                missing_edges.add((a,b))              
    else:
        continue

Wall time: 52 ms


In [13]:
len(missing_edges)
df_neg = pd.DataFrame(list(missing_edges), columns=['Author_A', 'Author_B'])
df_neg = df_neg.groupby(['Author_A','Author_B']).size().reset_index().rename(columns={0:'count'})
df_neg['Author_A'] = df_neg['Author_A'].astype(str)
df_neg['Author_B'] = df_neg['Author_B'].astype(str)
ng_neg = nx.convert_matrix.from_pandas_edgelist(df_neg, source= 'Author_A', target = 'Author_B', edge_attr= 'count')
print(df_neg)
df_pos = g_df
print(nx.info(ng_neg))
sum(df_pos.duplicated())
y_pos = np.ones(len(df_pos))
y_neg = np.zeros(len(df_pos))

      Author_A Author_B  count
0           32  1723356      1
1          254  1621177      1
2          326   707842      1
3          331   886105      1
4          583  1052150      1
...        ...      ...    ...
16031  1861887   910005      1
16032  1861978   240709      1
16033  1861987   610898      1
16034  1862072   811160      1
16035  1862115  1067592      1

[16036 rows x 3 columns]
Name: 
Type: Graph
Number of nodes: 31815
Number of edges: 16036
Average degree:   1.0081


In [14]:
feature_1_neg = []
for s,t in ng_neg.edges():
    try:
        metric = nx.common_neighbor_centrality(ng_neg, [(s,t)])
        for u,v,p in metric:
            feature_1_neg.append(p)
    except:
        feature_1_neg.append(0)

6362.999999999998


In [49]:
btwc = nx.edge_betweenness_centrality(ng)

In [87]:
btwc_neg = nx.edge_betweenness_centrality(ng_neg)

In [88]:
btwc_neg

{('127', '1290865'): 1.9790765887895743e-09,
 ('158', '1522908'): 1.9790765887895743e-09,
 ('265', '1234968'): 1.9790765887895743e-09,
 ('275', '1238233'): 1.9790765887895743e-09,
 ('426', '850601'): 1.9790765887895743e-09,
 ('826', '1226397'): 1.9790765887895743e-09,
 ('876', '1400749'): 1.9790765887895743e-09,
 ('894', '24931'): 1.9790765887895743e-09,
 ('1049', '1673136'): 1.9790765887895743e-09,
 ('1114', '134359'): 1.9790765887895743e-09,
 ('1200', '520604'): 1.9790765887895743e-09,
 ('1444', '185903'): 3.9581531775791485e-09,
 ('1444', '1942'): 3.9581531775791485e-09,
 ('1630', '1281951'): 1.9790765887895743e-09,
 ('1698', '169525'): 1.9790765887895743e-09,
 ('1985', '97697'): 1.9790765887895743e-09,
 ('2006', '417559'): 1.9790765887895743e-09,
 ('2054', '724620'): 1.9790765887895743e-09,
 ('2194', '608136'): 3.9581531775791485e-09,
 ('608136', '1359668'): 3.9581531775791485e-09,
 ('2514', '254354'): 1.9790765887895743e-09,
 ('2609', '328862'): 1.9790765887895743e-09,
 ('2618', '

In [89]:
df_neg

,Author_A,Author_B,count
0,127,1290865,1
1,158,1522908,1
2,265,1234968,1
3,275,1238233,1
4,426,850601,1
...,...,...,...
16031,1861339,679127,1
16032,1861348,201465,1
16033,1861492,1303111,1
16034,1861897,1723381,1


In [90]:
feature_2_neg = []
for s,t in ng_neg.edges():
    try:
        feature_2_neg.append(btwc_neg[(str(s),str(t))])
    except:
        feature_2_neg.append(0) 

In [83]:
feature_2 = []
for s,t in ng.edges():
    try:
        feature_2.append(btwc[(str(s),str(t))])
    except:
        feature_2.append(0) 

In [91]:
print(feature_2_neg)

[1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 3.9581531775791485e-09, 3.9581531775791485e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 3.9581531775791485e-09, 3.9581531775791485e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895743e-09, 1.9790765887895

In [17]:
feature_3 = []
for s,t in ng.edges():
    try:
        metric = nx.adamic_adar_index(ng, [(s,t)])
        for u,v,p in metric:
            feature_3.append(p)
    except:
        feature_3.append(0)

In [20]:
# only returns 0 for negative graph.
feature_3_neg = []
for s,t in ng_neg.edges():
    try:
        metric = nx.adamic_adar_index(ng_neg, [(s,t)])
        for u,v,p in metric:
            feature_3_neg.append(p)
    except:
        feature_3_neg.append(0)

In [53]:
cc = nx.closeness_centrality(ng)
feature_4 = []
for s,t in ng.edges():
    try:
        feature_4.append(cc[str(s)])
    except:
        feature_4.append(0)

In [92]:
cc_neg = nx.closeness_centrality(ng_neg)
feature_4_neg = []
for s,t in ng_neg.edges():
    try:
        feature_4_neg.append(cc_neg[str(s)])
    except:
        feature_4_neg.append(0) 

In [93]:
dc_neg = nx.degree_centrality(ng_neg)
feature_5_neg = []
for s,t in ng_neg.edges():
    try:
        feature_5_neg.append(dc_neg[str(s)])
    except:
        feature_5_neg.append(0)

In [56]:
dc = nx.degree_centrality(ng)
feature_5 = []
for s,t in ng.edges():
    try:
        feature_5.append(dc[str(s)])
    except:
        feature_5.append(0)

In [ ]:
# need "community" information for that. Do we have that piece of information?
wic = nx.within_inter_cluster(ng)

In [21]:
feature_6 = []
for s,t in ng.edges():
    try:
        metric = nx.jaccard_coefficient(ng, [(s,t)])
        for u,v,p in metric:
            feature_6.append(p)
    except:
        feature_6.append(0)

In [23]:
# returns 0 for all edges.
feature_6_neg = []
for s,t in ng_neg.edges():
    try:
        metric = nx.jaccard_coefficient(ng_neg, [(s,t)])
        for u,v,p in metric:
            feature_6_neg.append(p)
    except:
        feature_6_neg.append(0)

In [30]:
feature_7 = []
for s,t in ng.edges():
    try:
        metric = nx.preferential_attachment(ng, [(s,t)])
        for u,v,p in metric:
            feature_7.append(p)
    except:
        feature_7.append(0)

In [32]:
feature_7_neg = []
for s,t in ng_neg.edges():
    try:
        metric = nx.preferential_attachment(ng_neg, [(s,t)])
        for u,v,p in metric:
            feature_7_neg.append(p)
    except:
        feature_7_neg.append(0)

In [ ]:
pr = nx.pagerank(ng)
# pagerank Author_A.
for s,t in ng.edges():
    try:
        rank = pr[s]
        feature_8.append(rank)
    except:
        feature_8.append(0)

In [ ]:
# pagerank Author_B.
for s,t in ng.edges():
    try:
        rank = pr[t]
        feature_9.append(rank)
    except:
        feature_9.append(0)

In [ ]:
pr = nx.pagerank(ng_neg)
# pagerank Author_A neg edges.
for s,t in ng_neg.edges():
    try:
        rank = pr[s]
        feature_8_neg.append(rank)
    except:
        feature_8_neg.append(0)

In [ ]:
# pagerank Author_B neg edges.
for s,t in ng_neg.edges():
    try:
        rank = pr[t]
        feature_9_neg.append(rank)
    except:
        feature_9_neg.append(0)

In [ ]:
katz = nx.katz.katz_centrality(ng)
# Katz centrality Author_A
for s,t in ng.edges():
    try:
        kc = katz[s]
        feature_10.append(kc)
    except:
        feature_10.append(0)

In [ ]:
# Katz centrality Author_B
for s,t in ng.edges():
    try:
        kc = katz[t]
        feature_11.append(kc)
    except:
        feature_11.append(0)

In [ ]:
katz = nx.katz.katz_centrality(ng_neg)
# Katz centrality Author_A neg edges.
for s,t in ng_neg.edges():
    try:
        kc = katz[s]
        feature_10_neg.append(kc)
    except:
        feature_10_neg.append(0)

In [ ]:
# Katz centrality Author_B neg edges.
for s,t in ng_neg.edges():
    try:
        kc = katz[t]
        feature_11_neg.append(kc)
    except:
        feature_11_neg.append(0)

In [ ]:
hits = nx.hits(ng)
# for Author A.
for s,t in ng.edges():
    try:
        hit = hits[s]
        for i,j in hit:
            feature_12.append(i)
            feature_13.append(j)
    except:
        feature_12.append(0)
        feature_13.append(0)

In [ ]:
# for Author B.
for s,t in ng.edges():
    try:
        hit = hits[t]
        for i,j in hit:
            feature_14.append(i)
            feature_15.append(j)
    except:
        feature_14.append(0)
        feature_15.append(0)

In [ ]:
hits = nx.hits(ng_neg)
# for Author A neg edges.
for s,t in ng_neg.edges():
    try:
        hit = hits[s]
        for i,j in hit:
            feature_12_neg.append(i)
            feature_13_neg.append(j)
    except:
        feature_12_neg.append(0)
        feature_13_neg.append(0)

In [ ]:
# for Author B neg edges.
for s,t in ng.edges():
    try:
        hit = hits[t]
        for i,j in hit:
            feature_14_neg.append(i)
            feature_15_neg.append(j)
    except:
        feature_14_neg.append(0)
        feature_15_neg.append(0)

In [94]:
print(len(feature_1))
df_pos['cnc'] = feature_1
df_pos['btwc'] = feature_2
df_pos['adar'] = feature_3
df_pos['cc'] = feature_4
df_pos['dc'] = feature_5
df_pos['jc'] = feature_6
df_pos['pa'] = feature_7
df_pos['pagerank_A'] = feature_8
df_pos['pagerank_B'] = feature_9
df_pos['katz_A'] = feature_10
df_pos['katz_B'] = feature_11
df_pos['hits_authority_A'] = feature_12
df_pos['hits_hubs_A'] = feature_13
df_pos['hits_authority_B'] = feature_14
df_pos['hits_hubs_B'] = feature_15
df_neg['cnc'] = feature_1_neg
df_neg['btwc'] = feature_2_neg
df_neg['adar'] = feature_3_neg
df_neg['cc'] = feature_4_neg
df_neg['dc'] = feature_5_neg
df_neg['jc'] = feature_6_neg
df_neg['pa'] = feature_7_neg
df_neg['pagerank_A'] = feature_8_neg
df_neg['pagerank_B'] = feature_9_neg
df_neg['katz_A'] = feature_10_neg
df_neg['katz_B'] = feature_11_neg
df_neg['hits_authority_A'] = feature_12_neg
df_neg['hits_hubs_A'] = feature_13_neg
df_neg['hits_authority_B'] = feature_14_neg
df_neg['hits_hubs_B'] = feature_15_neg

16036


In [95]:
df_final_train = df_pos.append(df_neg,ignore_index=True)
y_final_train = np.concatenate((y_pos,y_neg))

In [96]:
df_final_train

,Author_A,Author_B,count,cnc,btwc,cc,dc
0,0,1236,14,0,2.013822e-04,0.187879,0.002124
1,0,1655,9,0,6.067351e-05,0.187879,0.002124
2,0,1797,4,0,1.644754e-07,0.187879,0.002124
3,0,2414,1,0,4.159867e-06,0.187879,0.002124
4,0,2568,8,0,6.390372e-05,0.187879,0.002124
...,...,...,...,...,...,...,...
32067,1861339,679127,1,0,1.979077e-09,0.000031,0.000031
32068,1861348,201465,1,0,1.979077e-09,0.000031,0.000031
32069,1861492,1303111,1,0,1.979077e-09,0.000031,0.000031
32070,1861897,1723381,1,0,1.979077e-09,0.000031,0.000031
